In [2]:
import pandas as pd


In [ ]:
def create_emart_shopping_prompt(df, user_location, user_budget, user_preferences, region_name="사용자 지역"):
    # 데이터 전처리
    df_clean = df.copy()
    df_clean = df_clean.dropna(subset=["상품명", "판매가"])
    df_clean["판매가"] = df_clean["판매가"].str.extract(r"(\d[\d,]*)")[0].str.replace(",", "")
    df_clean["판매가"] = pd.to_numeric(df_clean["판매가"], errors="coerce")
    df_clean = df_clean.dropna(subset=["판매가"])

    # top 10만 선택 (너무 많으면 프롬프트 과다)
    df_top = df_clean.sort_values(by="판매가").head(10)

    # markdown 스타일 텍스트로 변환
    item_lines = []
    for _, row in df_top.iterrows():
        item_lines.append(f"- {row['상품명']} | {int(row['판매가'])}원 | 정상가: {row['정상가'] or '정보 없음'} | 할인율: {row['할인율'] or '정보 없음'}")

    item_block = "\n".join(item_lines)

    # 프롬프트 템플릿 구성
    prompt = f"""
당신은 지역 마트의 과일 세일 정보를 분석하여, 사용자에게 예산 내에서 효율적인 장보기 계획을 제안하는 GPT입니다.

 사용자 정보:
- 위치: {user_location}
- 예산: {user_budget}원
- 선호 과일: {user_preferences or '특별히 없음'}

 {region_name} 내 주요 과일 세일 품목 (상위 10개):
{item_block}

 조건:
1. 예산을 초과하지 않고, 가성비 좋고 다양한 과일을 구매할 수 있게 조합하세요.
2. 할인율이 높은 상품을 우선 추천하세요.
3. 동일 품목 중복 없이, 총 예상 금액을 명시하고 예산과의 차이도 알려주세요.
4. 결과는 간결한 표 형식으로 구성하고, 추천 요약도 간단히 포함하세요.

이 정보를 기반으로 최적의 장보기 조합을 제안하세요.
"""
    return prompt


In [5]:
# 1. CSV 파일 불러오기
file_path = "11emart_flyer_fruits_filtered.csv"
df = pd.read_csv(file_path)

In [6]:
# 3. 사용자 정보 설정 및 프롬프트 생성
user_location = "서울시 성동구"
user_budget = 15000
user_preferences = "딸기, 키위, 감귤"

generated_prompt = create_emart_shopping_prompt(
    df,
    user_location,
    user_budget,
    user_preferences
)

In [7]:
import os
from openai import OpenAI

# API 키 설정 (환경 변수에 저장된 값 사용)
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# GPT에게 프롬프트 전달
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "당신은 마트 장보기 추천 전문가입니다."},
        {"role": "user", "content": generated_prompt}
    ],
    temperature=0.7
)

# 결과 출력
print(response.choices[0].message.content)

🛒 추천 장보기 리스트:

| 품목 | 가격 | 할인율 |
| --- | --- | --- |
| 친환경 생 블루베리 (200g/팩) | 6980원 | 30% |
| 제스프리 골드키위 점보 (10개) | 12800원 | - |

📝 요약: 
서울시 성동구의 과일 세일 정보를 분석한 결과, 친환경 생 블루베리와 제스프리 골드키위 점보를 추천드립니다. 두 품목의 총 가격은 19,780원으로, 예산인 15,000원보다 4,780원 초과하였습니다. 

하지만 두 품목 모두 사용자가 선호하는 과일 리스트에 포함되어 있으며, 특히 블루베리는 30%의 높은 할인율을 적용받고 있습니다. 따라서 이번에는 예산을 조금 초과하더라도, 다양한 과일을 경험해 보실 수 있는 좋은 기회라고 생각합니다. 

다음에는 예산을 좀 더 넉넉하게 잡아, 선호하는 다양한 과일을 맛볼 수 있도록 계획해보시는 것은 어떨까요?

